# EE4745 Neural Network Final Project - Master Notebook
## Defending LSU's Sports AI - Complete Implementation

**Problems Covered:**
- Problem A: Sports Image Classification (220 pts)
- Problem B: Adversarial Attacks (100 pts)
- Problem C: Model Compression (100 pts)

**⏱️ Estimated Runtime:** 2-3 hours (depending on epochs)

---

## 📋 Setup Instructions

**Before running this notebook, you need to:**

1. **Upload the dataset to Google Drive** (recommended) or upload directly
2. **Have your GitHub repository ready** (https://github.com/Tyler-Trauernicht/Neural-Final.git)

**Dataset Location Options:**
- Option A: Upload `EE4745-project-data-to-release.zip` to Google Drive
- Option B: Upload dataset directly in the "Upload Dataset" cell below

---

## 🔧 STEP 1: Environment Setup

In [ ]:
# Check GPU availability (optional - project works on CPU)
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
else:
    print("Running on CPU (as designed for this project)")

## 📁 STEP 2: Mount Google Drive (Option A - Recommended)

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")
print("\nPlease ensure your dataset is at:")
print("  /content/drive/MyDrive/EE4745-project-data-to-release/")
print("  (with 'train' and 'valid' folders inside)")

## 📦 STEP 3: Clone Repository

In [ ]:
# Clone the repository from GitHub
!git clone https://github.com/Tyler-Trauernicht/Neural-Final.git
%cd Neural-Final

# Verify we're in the right directory
!pwd
!ls -la

print("\n✅ Repository cloned successfully!")

## 📚 STEP 4: Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch torchvision torchaudio
!pip install -q numpy pandas matplotlib seaborn
!pip install -q tensorboard scikit-learn tqdm
!pip install -q Pillow opencv-python

print("✅ All dependencies installed!")

## 🔗 STEP 5: Setup Dataset Link

In [ ]:
import os

# OPTION A: If dataset is in Google Drive
# Adjust this path to where YOUR dataset is located in Google Drive
drive_dataset_path = '/content/drive/MyDrive/EE4745-project-data-to-release'

# Create symbolic link
if os.path.exists(drive_dataset_path):
    !ln -s {drive_dataset_path} data
    print(f"✅ Dataset linked from Google Drive: {drive_dataset_path}")
else:
    print(f"❌ Dataset not found at: {drive_dataset_path}")
    print("\nPlease either:")
    print("1. Upload dataset to Google Drive at the path above, OR")
    print("2. Use OPTION B below to upload directly")

# Verify dataset structure
if os.path.exists('data'):
    print("\n📊 Dataset structure:")
    !ls -la data/
    print("\nTraining classes:")
    !ls data/train/ 2>/dev/null || echo "Train folder not found"
    print("\nValidation classes:")
    !ls data/valid/ 2>/dev/null || echo "Valid folder not found"

## 📤 OPTION B: Upload Dataset Directly (Alternative)

In [ ]:
# ONLY RUN THIS IF YOU DIDN'T USE GOOGLE DRIVE
# This will prompt you to upload the dataset ZIP file

from google.colab import files
import zipfile

print("📤 Upload your EE4745-project-data-to-release.zip file:")
uploaded = files.upload()

# Extract the uploaded ZIP
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"\n📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        
        # Find the extracted folder and link it
        !ln -s EE4745-project-data-to-release data
        print("✅ Dataset extracted and linked!")

# Verify
!ls -la data/

## ✅ STEP 6: Verify Setup

In [ ]:
# Test dataset loading
import sys
sys.path.append('/content/Neural-Final')

from src.dataset.sports_dataset import SportsDataset, get_dataloaders

print("🧪 Testing dataset loading...\n")

try:
    train_loader, val_loader, num_classes = get_dataloaders(
        data_dir='data',
        batch_size=32,
        image_size=32,
        num_workers=2
    )
    print("\n✅ Dataset loaded successfully!")
    print(f"Number of classes: {num_classes}")
    print(f"Classes: {SportsDataset.CLASSES}")
    
    # Test loading a batch
    images, labels = next(iter(train_loader))
    print(f"\nBatch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print("\n🎉 All setup complete! Ready to train!")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\nPlease check:")
    print("1. Dataset is correctly uploaded/linked")
    print("2. Dataset has 'train' and 'valid' folders")
    print("3. Each folder contains the 10 sports class folders")

---
# 🎯 PROBLEM A: Sports Image Classification
---

## 🚀 Train SimpleCNN Model

In [ ]:
# Train SimpleCNN with reduced epochs for faster training
# Adjust --epochs based on your available time
# For quick testing: --epochs 5
# For full training: --epochs 50

!python train_problem_a.py \
    --model SimpleCNN \
    --epochs 20 \
    --batch_size 32 \
    --learning_rate 0.001 \
    --device cpu \
    --image_size 32

print("\n✅ SimpleCNN training complete!")

## 🚀 Train ResNetSmall Model

In [ ]:
# Train ResNetSmall
# Note: ResNet may take longer due to more parameters

!python train_problem_a.py \
    --model ResNetSmall \
    --epochs 20 \
    --batch_size 16 \
    --learning_rate 0.001 \
    --device cpu \
    --image_size 32

print("\n✅ ResNetSmall training complete!")

## 📊 Generate Interpretability Analysis

In [ ]:
# Generate Saliency Maps and Grad-CAM visualizations
# This will analyze both correct and misclassified examples

!python train_problem_a.py \
    --model both \
    --skip_training \
    --interpretability_samples 15 \
    --analyze_misclassifications

print("\n✅ Interpretability analysis complete!")

## 📈 View Problem A Results

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

# Display training curves
print("📊 Training Curves:")
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

for idx, model_name in enumerate(['SimpleCNN', 'ResNetSmall']):
    curve_path = f'results/problem_a/training_curves/{model_name}_training_curves.png'
    if os.path.exists(curve_path):
        img = Image.open(curve_path)
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(model_name)

plt.tight_layout()
plt.show()

# Display confusion matrices
print("\n🎯 Confusion Matrices:")
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

for idx, model_name in enumerate(['SimpleCNN', 'ResNetSmall']):
    cm_path = f'results/problem_a/evaluation/{model_name}_confusion_matrix.png'
    if os.path.exists(cm_path):
        img = Image.open(cm_path)
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f'{model_name} Confusion Matrix')

plt.tight_layout()
plt.show()

# Display sample interpretability results
print("\n🔍 Sample Interpretability Results:")
interp_dir = 'results/problem_a/interpretability/SimpleCNN_saliency/'
if os.path.exists(interp_dir):
    sample_files = sorted(os.listdir(interp_dir))[:4]
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    for idx, fname in enumerate(sample_files):
        if fname.endswith('.png'):
            img = Image.open(os.path.join(interp_dir, fname))
            axes[idx//2, idx%2].imshow(img)
            axes[idx//2, idx%2].axis('off')
            axes[idx//2, idx%2].set_title(fname)
    plt.tight_layout()
    plt.show()

print("\n✅ Problem A complete! Check results/problem_a/ for all outputs.")

---
# 🛡️ PROBLEM B: Adversarial Attacks
---

## ⚔️ Generate Adversarial Examples

In [ ]:
# Run comprehensive adversarial attack analysis
# This includes FGSM, PGD, targeted/untargeted attacks

!python attack_problem_b.py \
    --data_dir data \
    --num_samples 20 \
    --device cpu

print("\n✅ Adversarial attacks complete!")

## 🔄 Transferability Analysis

In [ ]:
# Analyze how attacks transfer between models

!python attack_problem_b.py \
    --transferability_analysis \
    --detailed_interpretability

print("\n✅ Transferability analysis complete!")

## 📊 View Problem B Results

In [ ]:
# Display adversarial attack results
print("🛡️ Problem B Results:")

# Check for results files
prob_b_dir = 'results/problem_b/'
if os.path.exists(prob_b_dir):
    print(f"\n📁 Results saved in: {prob_b_dir}")
    !ls -la {prob_b_dir}
    
    # Display attack statistics if available
    import json
    stats_file = os.path.join(prob_b_dir, 'attack_statistics.json')
    if os.path.exists(stats_file):
        with open(stats_file, 'r') as f:
            stats = json.load(f)
        print("\n📊 Attack Statistics:")
        print(json.dumps(stats, indent=2))
else:
    print("⚠️ Results directory not found. Check if attacks ran successfully.")

print("\n✅ Problem B complete!")

---
# ⚡ PROBLEM C: Model Compression
---

## ✂️ Apply Unstructured Pruning

In [ ]:
# Apply pruning at 20%, 50%, 80% sparsity levels
# This will prune both models and fine-tune them

!python prune_problem_c.py \
    --sparsity_levels 0.2 0.5 0.8 \
    --fine_tune_epochs 10 \
    --fine_tune_lr 0.0001

print("\n✅ Model pruning complete!")

## 📊 Comprehensive Pruning Analysis

In [ ]:
# Run complete analysis including performance and robustness evaluation

!python complete_problem_c_analysis.py

print("\n✅ Pruning analysis complete!")

## 📈 View Problem C Results

In [ ]:
# Display pruning results and analysis
print("⚡ Problem C Results:")

prob_c_dir = 'results/problem_c/'
if os.path.exists(prob_c_dir):
    print(f"\n📁 Results saved in: {prob_c_dir}")
    
    # Display pruning analysis plots if available
    analysis_file = os.path.join(prob_c_dir, 'demo_pruning_analysis.png')
    if os.path.exists(analysis_file):
        print("\n📊 Pruning Analysis:")
        img = Image.open(analysis_file)
        plt.figure(figsize=(15, 8))
        plt.imshow(img)
        plt.axis('off')
        plt.title('Pruning Trade-off Analysis')
        plt.show()
    
    # Display results table if available
    import pandas as pd
    results_csv = os.path.join(prob_c_dir, 'demo_pruning_results.csv')
    if os.path.exists(results_csv):
        print("\n📊 Pruning Results Table:")
        df = pd.read_csv(results_csv)
        print(df.to_string(index=False))
else:
    print("⚠️ Results directory not found. Check if pruning ran successfully.")

print("\n✅ Problem C complete!")

---
# 📊 FINAL: Comprehensive Results Compilation
---

## 🎯 Generate Master Analysis

In [ ]:
# Compile all results into final analysis

!python results/final/run_final_analysis.py

print("\n✅ Final analysis complete!")
print("\n📁 All results compiled in: results/final/")

## 📊 View Master Performance Dashboard

In [ ]:
# Display the master performance dashboard

dashboard_path = 'results/final/figures/master_performance_dashboard.png'
if os.path.exists(dashboard_path):
    print("🎯 Master Performance Dashboard:")
    img = Image.open(dashboard_path)
    plt.figure(figsize=(20, 12))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Complete Project Performance Dashboard', fontsize=16)
    plt.show()
else:
    print("⚠️ Dashboard not found. Running analysis...")
    !python results/final/analysis/visualization_dashboard.py
    if os.path.exists(dashboard_path):
        img = Image.open(dashboard_path)
        plt.figure(figsize=(20, 12))
        plt.imshow(img)
        plt.axis('off')
        plt.show()

## 📋 View Executive Summary

In [ ]:
# Display executive summary

summary_path = 'results/final/summary/executive_summary.md'
if os.path.exists(summary_path):
    print("📄 Executive Summary:")
    print("=" * 80)
    with open(summary_path, 'r') as f:
        print(f.read())
    print("=" * 80)
else:
    print("⚠️ Executive summary not found.")

print("\n✅ All analyses complete!")

---
# 💾 DOWNLOAD ALL RESULTS
---

## 📥 Create Results Archive

In [ ]:
# Create comprehensive ZIP archive of all results

import shutil
from datetime import datetime

# Create timestamp for filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archive_name = f'EE4745_Neural_Final_Results_{timestamp}'

print("📦 Creating results archive...\n")

# Create ZIP of all results
!zip -r {archive_name}.zip \
    results/ \
    checkpoints/ \
    logs/ \
    notebooks/ \
    -x "*__pycache__*" "*.pyc"

print(f"\n✅ Archive created: {archive_name}.zip")
print(f"📊 Archive size:")
!ls -lh {archive_name}.zip

## 📥 Download Results

In [ ]:
# Download the results archive to your local machine

from google.colab import files

# Get the archive filename
import glob
archive_files = glob.glob('EE4745_Neural_Final_Results_*.zip')

if archive_files:
    archive_file = archive_files[0]
    print(f"📥 Downloading: {archive_file}")
    print("⏳ This may take a few minutes depending on file size...\n")
    
    files.download(archive_file)
    
    print("\n✅ Download complete!")
    print("\n📦 Your archive contains:")
    print("  - results/          All experimental results")
    print("  - checkpoints/      All trained model files")
    print("  - logs/            TensorBoard training logs")
    print("  - notebooks/       Jupyter analysis notebooks")
else:
    print("❌ No archive found. Please run the archive creation cell first.")

## 📄 Download Individual Components (Optional)

In [ ]:
# Download specific components if needed

print("📥 Available for individual download:\n")

# Option 1: Download only checkpoints
# !zip -r checkpoints_only.zip checkpoints/
# files.download('checkpoints_only.zip')

# Option 2: Download only final results
# !zip -r final_results.zip results/final/
# files.download('final_results.zip')

# Option 3: Download only notebooks
# !zip -r notebooks.zip notebooks/
# files.download('notebooks.zip')

print("Uncomment the sections above to download specific components.")

---
# 🎉 COMPLETION SUMMARY
---

In [ ]:
# Final completion summary

print("="*80)
print("🎓 EE4745 NEURAL NETWORK FINAL PROJECT - COMPLETE!")
print("="*80)
print("\n✅ Problem A: Sports Image Classification - COMPLETE")
print("   - SimpleCNN trained and evaluated")
print("   - ResNetSmall trained and evaluated")
print("   - Interpretability analysis generated")
print("   - Model comparison completed")

print("\n✅ Problem B: Adversarial Attacks - COMPLETE")
print("   - FGSM and PGD attacks implemented")
print("   - Targeted and untargeted attacks executed")
print("   - Transferability analysis completed")
print("   - Attack visualizations generated")

print("\n✅ Problem C: Model Compression - COMPLETE")
print("   - Unstructured pruning applied (20%, 50%, 80%)")
print("   - Performance analysis completed")
print("   - Robustness evaluation finished")
print("   - Trade-off visualizations created")

print("\n✅ Final Results Compilation - COMPLETE")
print("   - Master performance dashboard generated")
print("   - Executive summary created")
print("   - All results compiled and ready")

print("\n" + "="*80)
print("📦 DELIVERABLES READY FOR SUBMISSION")
print("="*80)
print("\n1. ✅ Source code (all .py files)")
print("2. ✅ Trained model checkpoints")
print("3. ✅ Experimental results and visualizations")
print("4. ✅ Analysis reports and summaries")
print("5. ✅ Jupyter notebooks")

print("\n📥 Download the archive above to get all materials!")
print("\n🎯 Next Step: Write your final report using the generated materials")
print("\n" + "="*80)
print("Good luck with your submission! 🚀")
print("="*80)